# HW1 SOLUTION : [YOUR_NAME | YOUR STUDENT ID]

### (80%) Perform basic data engineering on the light pollution dataset. 


For our data engineering we want to broadly 
complete the following:

1. combine all the GaN files into a single file and thus a
   single Pandas DataFrame,
2. convert specific data fields to numeric fields using
   Pandas,
3. combine date and time fields for easier time series evaluation,
3. split data out of specific fields and into new simpler
   fields,
4. use Pandas to understand the underlying distrubution
   of certain data.




**&#167; Task:**  **Use the file of URLs from the first assignment to load the data from 2014-2024 and store it all into a single file.**


In [ ]:
# CODE SOLUTION HERE

**&#167; Task:**  **Convert categorical fields to numeric ones.**

We will convert these to numeric values using the following guidance:

* "clear" &#8594; 0
* "1/4 of sky" &#8594; 0.25
* "1/2 of sky" &#8594; 0.50
* "over 1/2 of sky" &#8594; 0.75

In [ ]:
# CODE SOLUTION HERE

**&#167; Task:**  **Convert `Constellation` to numeric binary values (binarize the column).**


In [ ]:
# CODE SOLUTION HERE

**&#167; Task:**  **Create 5 binary columns `loc_urban`, `loc_suburban`, `loc_rural`, `loc_remote` and `loc_unknown`.**

In [ ]:
# CODE SOLUTION HERE

**&#167; Task:**  **Merge `LocalDate` and `LocalTime` and convert to a `datetime` object.**


In [ ]:
# CODE SOLUTION HERE

**&#167; Task:**  **Convert missing `SQMReading` values to `-1`.**

In [ ]:
# CODE SOLUTION HERE

**&#167; Task:**  **Store a new dataset file with a reduced set columns which include our numeric ones.** 

In [ ]:
# CODE SOLUTION HERE